## HIV News articles extraction from Khaama Press. Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for HIV News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'HIV site:www.khaama.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 46 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            headlines.append(article.title.strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            authors.append(soup.select_one('.author-single').a.text)
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            dates.append(article.meta_data['article']['published_time'].split('T')[0])
        except:
            dates.append(None)
            
        # Extracts the news category
        try:
            category.append(article.meta_data['category'])
        except:
            category.append(None)
            
        # Extracts the news articles
        try:
            news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(' '.join(article.summary.split()))
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 3min 49saama.com/persian/archives/748715/08/-youth-health-line-in-afghanistan-784/on-02060/118/1412/9/


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

46 46 46 46 46 46 46 46 46 46


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'D:\\#Backups\\Desktop\\!Code!\\CDRI\\HIV\\Data Extraction\\#Datasets\\'
    tbl.to_csv(path+'Khaama_Press.csv', index=False)
else:
    print('Array lenght does not match!')

tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Publication,Articles,category,Keywords,Summaries,Source_URLs
0,Drug ‘can greatly reduce risk of HIV infection’,The largest online news service for Afghanistan,Khaama Press,2010-11-25,The Khaama Press News Agency,A drug used to treat HIV-positive patients may...,{},"[drug, reduce, infection, results, used, men, ...",A drug used to treat HIV-positive patients may...,https://www.khaama.com/drug-can-greatly-reduce...
1,HIV Transmission,Some questions and answers on how HIV is trans...,None,None,None,Some groups of people in the United States are...,{},"[united, anal, sex, transmission, prevent, ris...","Risky behaviors, like having anal or vaginal s...",https://www.cdc.gov/hiv/basics/transmission.html
2,,,None,None,None,,{},[],,http://townofstratford.com/filestorage/1302/40...
3,HIV/AIDS - Symptoms and causes,,None,None,Mayo Clinic,Overview Acquired immunodeficiency syndrome (A...,{},"[causes, blood, common, infection, aids, body,...","By damaging your immune system, HIV interferes...",https://www.mayoclinic.org/diseases-conditions...
4,Afghanistan recorded 1367 HIV/AIDS positive ca...,The largest online news service for Afghanistan,Ahmad Shah Ghanizada,2013-07-06,The Khaama Press News Agency,"At least 1,367 positive cases of Human immunod...",{},"[afghanistan, paikan, positive, 2011, virus, h...","At least 1,367 positive cases of Human immunod...",https://www.khaama.com/afghanistan-recorded-13...


In [9]:
tbl.shape

(46, 10)